# MasterChessMind - Project Intro
What are some of the most significates dates in the history of computer science?

This notebook was created by Yuval Krinsky as part of the Data Science workshop at the Open University of Israel, Spring 

## Chess in the advancements of Artificial Intelligence
Throughout history, chess was considered an indicator of wisdom and one who was good at it, was regarded as a smart and wise man. 
These days, chess is one of the world's most popular board games, played and admired for its strategic depth and intellectual challenge.

Chess players have dedicated years and decades in improving their strategies and tactics in the game. 
And as chess was considered a test of intellectual superiority, it's no suprise computer scientists took on the challenge of developing a computer program which will be able to play and defeat the best of the best human players.

Therefore, when in 1997, the **Deep Blue super computer defeated the current world champion and the chess legend Garry Kasparov**, it was and still considered a major milestone in the development of the field of AI in computer science.
The news of the defeat spread like fire in the whole world, and Deep Blue is even the subject of a few movies and books.
It important to mention that the task of building Deep Blue was sure not a piece of cake, and the computer giant IBM has invested more than 10 years in it's development!

![AI Timeline](./res/AI_timeline.png)

## Personal Connection
I myself is a bit of chess fan, and doesn't miss the chance to play against a friend or a family member.
I also used my love for the game to teach volenturylly children the game, to empower them through the game, with a non-profit organization called cpchess.

And from a personal perspective, I withstand the complexity of the game and the high skills needed to be a good player.
Therefore I thought of this as a great example to show the power of AI and specifically ML.

## Project Goal
The project goal is to provide a smart AI assistant for chess games. Currently, I don't have a real use of it in the world, but it's a very good demonstration of the advancements we can use AI and ML for.

The project is divided into 2 big parts:
### Chess Computer Vision
The first part of the program will be responsible for analyzing an image of a chess board and giving as an output a formal representation which can be understood easily by the computer and will be used in the second part of the project.
### Position Evaluation
The second part of the project will be to evaluate all the different moves the player can do from this position, and give the user a very good move he can do.

Both parts are very hard and complex to develop without using ML, and I hope that by using ML tools, we could develop a much simpler solution which will still give good results.

***

# Developing Position Evaluation
As a reminder, this part of the project goal is - 

**Given a board position, evaluate which player is leading and by how much!**

This is probably the most complex and difficult problem in chess, as there are many complex factors which should be taken in to consideration in such evaluation.

And the **proposed solution for this problem is building a deep neural network, which will take as an input the board position, and will return the estimated evaluation**.

## Evaluation Intro
There is a standard representation for chess board evaluations. Given a chess position, most chess engines given a number which indicates who leads and by how much.

If the white player is leading, this number will be positive and sometime explicitly marked with a '+' as a prefix. If the black player is leading, the evaluation will be negative and will start with '-'.

Generally, different pieces are worth different points for evaluating. For example, a pawn (the most basic piece) is generally worth 1 point, while the rook a much stronger piece worth 5 points, and the queen, which is the most important piece except the king, is worth 9 points.
This is just the most basic idea for evaluating. In practice, many more factors should be factored in, such as the pieces positions, the positions related to other pieces, relation to other pieces remaining on the board and more.

But to summarize, an evaluation of '+1.42' means the white player is in the lead but not in such a big gap. And an evaluation of '-9.12' means the black player lead in a big way, and the game is pretty much over from here.

![Pieces Evaluations](./res/pieces_evaluations.png)

## Exploratory Data Analysis
I've used Kaggle and found a very big [dataset](https://www.kaggle.com/datasets/ronakbadhe/chess-evaluations), which maps board positions to their corresponding evaluation as calculated by Stockfish (a famous and highly complex open source chess engine).

From an inspection of the dataset 3 problems arose:
* Board representation
* Checkmate positions
* Overweighed data

### Board Representation
Our dataset looks like this:
```Csv
N1bk2nr/pp1p1ppp/8/4P3/8/8/P1qNPPPP/R3KB1R w KQ - 0 12,-409
r2k1bnr/2Np1ppp/b1n1p3/8/4P3/P4N2/1PP2PPP/R1B1K2R w KQ - 3 12,+382
8/6k1/pp1p3p/3P2p1/2PN1rP1/1P6/P6P/6K1 w - - 1 39,+431
```
The first field is the board representation, and the second value is the board evaluation (in centipawns units, which means every pawn is equal to 100 and not 1).

As you can clearly see, the board representation isn't something we can directly use in building a neural network.
This representation is actually a standard representation called FEN. FEN (Forsyth-Edwards Notation) is a concise text-based format used to describe the current state of a chessboard position, it includes the pieces positions and more advanced parts of the game which are needed to exactly describe the current state of the game.

We need to convert this representation to something we could use as the input layer for our neural network.
After learning and understanding this notation, I chose the important features for evaluating the board position (pieces placement and castling rights), and wrote a function that given a FEN string, builds a PyTorch tensor representing the board.

A chess board consists of 64 (8*8) squares, and we have 12 different pieces types (6 white and 6 black). Therefore I chose to represent the pieces placement with a long array of boolean bits. For each piece type, we have 64 bits, indicating if the piece is currently on the corresponding square.
For castling rights, I used 4 bits. And for which player turn it right now, one more bit.
In total, we have 773 (64*12+4+1) bits for representing the board.

In [1]:
import torch

def fen_to_bitboard_tensor(fen: str):
    # Initialize bitboards for different piece types
    empty = 0
    white_pawns = 0
    white_knights = 0
    white_bishops = 0
    white_rooks = 0
    white_queens = 0
    white_kings = 0
    black_pawns = 0
    black_knights = 0
    black_bishops = 0
    black_rooks = 0
    black_queens = 0
    black_kings = 0

    # Split FEN into parts
    fen_board_part, fen_turn_part, fen_castling_part, *_ = fen.split()

    # Translate FEN board representation to bitboards

    # Start from the 8th rank and a-file (0-based index)
    row = 7  
    col = 0 

    for char in fen_board_part:
        if char == '/':
			# Continue to next line
            row -= 1
            col = 0
        elif char.isnumeric():
        	# Numeric means empty spaces, so we skip accordingly to the next piece in row.
            col += int(char)
        else:
        	# Convert rank and file to a square index
            square = row * 8 + col 

            if char == 'P':
                white_pawns |= 1 << square
            elif char == 'N':
                white_knights |= 1 << square
            elif char == 'B':
                white_bishops |= 1 << square
            elif char == 'R':
                white_rooks |= 1 << square
            elif char == 'Q':
                white_queens |= 1 << square
            elif char == 'K':
                white_kings |= 1 << square
            elif char == 'p':
                black_pawns |= 1 << square
            elif char == 'n':
                black_knights |= 1 << square
            elif char == 'b':
                black_bishops |= 1 << square
            elif char == 'r':
                black_rooks |= 1 << square
            elif char == 'q':
                black_queens |= 1 << square
            elif char == 'k':
                black_kings |= 1 << square

            col += 1

    pieces = [
    	white_pawns, white_knights, white_bishops, white_rooks, white_queens, white_kings, 
    	black_pawns, black_knights, black_bishops, black_rooks, black_queens, black_kings
    ]
    pieces_board_bits = []
    for piece in pieces:
    	# Pad to 64 bits for each piece and skip the '0b' prefix
    	piece_bits_str = bin(piece)[2:].zfill(64)
    	piece_bits = [int(bit) for bit in piece_bits_str]

    	pieces_board_bits.extend(piece_bits)

    # Determine player turn
    turn_bits = [1 if fen_turn_part == 'w' else 0]

    # Determine castling rights
    in_castling = lambda x: 1 if x in fen_castling_part else 0
    white_kingside_castle = in_castling('K')
    white_queenside_castle = in_castling('Q')
    black_kingside_castle = in_castling('k')
    black_queenside_castle = in_castling('q')

    castling_bits = [white_kingside_castle, white_queenside_castle, black_kingside_castle, black_queenside_castle]

    full_board_bits = pieces_board_bits + turn_bits + castling_bits
    full_board_tensor = torch.tensor(full_board_bits, dtype=torch.float)

    return full_board_tensor

### Overweighed Data
Another problem that arise from our dataset can be seen in these examples:
```CSV
8/6k1/pp1p3p/3P2p1/2PN1rP1/1P6/P6P/6K1 w - - 1 39,+431
6R1/5p2/5k1P/8/4BKPn/r7/3p4/8 w - - 0 57,+7019
6R1/5p1P/5k2/8/4BKPn/r7/3p4/8 b - - 0 57,+6056
```

In some of the positions, one of the players is in a very big lead. These results could impact our training of the neural network much greater than what we want. 

For example, in one of the above examples, white leads in a 70 points! In a chess game, almost every player will win the game when he's up at even 15 points.

Therefore to minimize the affect of these big leads positions, we limit their values in the dataset to +15 or -15 (depends on if white/black is leading).


### Checkmate Positions 
2 more problems that arose from our dataset can be seen in these examples:
```CSV
8/6k1/pp1p3p/3P2p1/2PN1rP1/1P6/P6P/6K1 w - - 1 39,+431
6R1/5p2/5k1P/8/4BKPn/r7/3p4/8 w - - 0 57,+7019
6R1/5p1P/5k2/8/4BKPn/r7/3p4/8 b - - 0 57,+6056
4Q1k1/2q2pp1/2b2b1p/p4B2/1pN5/1P5P/P4PP1/4R1K1 b - - 0 30,#+1,c6e8
3N1k2/3R4/1r5p/4p1p1/5p2/2P5/2P2PPP/6K1 b - - 0 27,#-2,b6b1
```

In [6]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv(r'./res/chess_eval/tactic_evals.csv')

# Skip checkmate positions
df = df.loc[~df['Evaluation'].str.contains('#')]

df['Evaluation'] = df['Evaluation'].astype(int)
print(df)
very_big_lead_positions = df.query('Evaluation < -1500 or Evaluation > 1500')
print(very_big_lead_positions)
big_lead_percentage = len(very_big_lead_positions) / len(df)
print(f"Big lead positions has {len(very_big_lead_positions)} data points, and are {big_lead_percentage} of all dataset")



                                                       FEN  Evaluation  Move
4        6k1/pp6/3p4/2p1p3/2P1P1q1/1P1P2pP/P5P1/5K2 w -...         408  h3g4
5        6k1/pp6/3p4/2p1p3/2P1P1bq/1P1P2pP/P3Q1P1/5K2 w...         444  e2g4
6        6k1/pp6/3p4/2p1p3/2P1P1Qq/1P1P2pP/P5P1/5K2 b -...         428  h4g4
10       2k4r/pp3pp1/3N1p2/2pP1P1p/b3r1P1/P1P4P/2P5/R1R...         667  c8c7
12       6k1/pp6/3p4/2p1p3/2P1P1P1/1P1P2p1/P5P1/5K2 b -...         462  g8f7
...                                                    ...         ...   ...
2628214  3r4/3pk3/6q1/Qp1pPpP1/1PbPn3/4B3/5RKP/8 w - - ...       -1053  a5a3
2628215  r2qkbnr/5ppp/p1p1p3/3p4/3P2PP/2N5/PPb1QP2/R1B1...         215  h7h5
2628216  Q5b1/6pk/3q1pNp/1p1p3N/3P1P2/2P5/1P3KPP/8 b - ...         944  h7g6
2628217    3rb3/ppk1r2p/6p1/2P5/4B3/P7/6PP/3Q3K w - - 0 35        -692  d1c2
2628218  r6k/p1R2Qp1/1pP4p/3p4/3b1PPq/B6P/P5K1/8 w - - ...       -1138  f7g7

[2345781 rows x 3 columns]
                                                